# **Auto Reconnect to RSTP Stream**

#### **In this lesson we'll learn how to use a class to enable auto reconnection to a video stream**

In [2]:
import cv2
import requests
import time

class VideoCapture:
    def __init__(self, cam_address, cam_force_address=None, blocking=False):
        """
        cam_address: 카메라 피드의 IP 주소
        cam_force_address: 다른 클라이언트를 연결 해제하는 IP 주소 (강제 점유)
        blocking: true인 경우 read() 및 connect_camera() 메서드가 IP 카메라 재연결까지 차단됨
        """
        self.cam_address = cam_address
        self.cam_force_address = cam_force_address
        self.blocking = blocking
        self.capture = None
        
        # 주기적으로 재연결 시도 시간 설정 (초 단위)
        self.RECONNECTION_PERIOD = 0.5
        # 카메라 연결 메서드 호출
        self.connect_camera()

    def connect_camera(self):
        print("Connecting...")
        while True:
            try:
                if self.cam_force_address is not None:
                    requests.get(self.cam_force_address)

                self.capture = cv2.VideoCapture(self.cam_address)

                if not self.capture.isOpened():
                    time.sleep(self.RECONNECTION_PERIOD)
                    raise Exception("Could not connect to a camera: {0}".format(self.cam_address))

                print("Connected to a camera: {}".format(self.cam_address))

                break
            except Exception as e:
                print(e)

                if self.blocking is False:
                    break

                time.sleep(self.RECONNECTION_PERIOD)

    def getStream(self):
        """
        프레임을 읽고 프레임을 받지 못하면 카메라 재연결 시도

        :return: ret - 프레임이 성공적으로 읽혔는지 여부를 나타내는 부울 값
                 frame - 카메라에서 받은 OpenCV 이미지
        """

        ret, frame = self.capture.read()

        # 피드가 다운되면 재연결 시도
        if ret is False:
            self.connect_camera()

        return ret, frame

In [ ]:
# RTSP 스트림 URL을 사용하여 VideoCapture 객체 생성
cap = VideoCapture("rtsp://210.99.70.120:1935/live/cctv007.stream")

while(1):
    ret, frame = cap.getStream()
    
    if not ret or frame is None:
        print("Failed to receive frame")
        continue

    # 프레임 크기 자동 조정
    frame_height, frame_width = frame.shape[:2]

    # 창 크기 조정
    cv2.namedWindow('RTSP Stream', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('RTSP Stream', frame_width, frame_height)
    
    # 프레임을 화면에 표시
    cv2.imshow('RTSP Stream', frame)
    
    if cv2.waitKey(1) == 13:  # 13은 Enter 키입니다.
        print("Exited...")
        break
# 카메라를 해제하고 창을 닫습니다.
cap.capture.release()
cv2.destroyAllWindows()

Connecting...
Connected to a camera: rtsp://210.99.70.120:1935/live/cctv007.stream
Connecting...
Connected to a camera: rtsp://210.99.70.120:1935/live/cctv007.stream
Failed to receive frame
